In [1]:
import hipscat_import.pipeline as runner
from hipscat.pixel_math.hipscat_id import HIPSCAT_ID_COLUMN
from hipscat_import.catalog.arguments import ImportArguments

from paths import *

In [2]:
output_artifact_name = "sfd_multiorder_map"

### Importing

In [3]:
%%time

OUTPUT_DIR.mkdir(exist_ok=True, parents=True)

args = ImportArguments(
    input_path=str(PARQUET_DIR),
    input_format="parquet",
    use_hipscat_index=True,
    add_hipscat_index=False,
    output_artifact_name=output_artifact_name,
    sort_columns=HIPSCAT_ID_COLUMN,
    output_path=OUTPUT_DIR,
    pixel_threshold=1 << 20,
    highest_healpix_order=6,
    dask_n_workers=12,
    progress_bar=True,
    overwrite=True,
    resume=False,
)
runner.pipeline(args)

Finishing : 100%|██████████| 6/6 [00:01<00:00,  3.18it/s]


CPU times: user 38 s, sys: 17.3 s, total: 55.3 s
Wall time: 13min 47s


### Validation

In [4]:
import numpy as np
import pandas as pd
from hipscat.pixel_math.hipscat_id import hipscat_id_to_healpix

df = pd.read_parquet('data/output/sfd_multiorder_map/Norder=4/Dir=0/Npix=333.parquet')
df.set_index(HIPSCAT_ID_COLUMN, inplace=True)
display(df)

offsets = np.diff(df.index.values)
np.testing.assert_array_equal(
    hipscat_id_to_healpix(df.index, target_order=df['pixel_Norder']),
    df['pixel_Npix'],
)

,pixel_Norder,pixel_Npix,ebv,Norder,Dir,Npix
_hipscat_index,,,,,,
1499698675914375168,12,21823488,0.021717,4,0,333
1499698744633851904,12,21823489,0.021764,4,0,333
1499698813353328640,12,21823490,0.021797,4,0,333
1499698882072805376,12,21823491,0.021838,4,0,333
1499698950792282112,12,21823492,0.021766,4,0,333
...,...,...,...,...,...,...
1504202202527301632,14,350224367,0.016499,4,0,333
1504202206822268928,13,87556092,0.016498,4,0,333
1504202224002138112,13,87556093,0.016441,4,0,333
